In [67]:
import os
import pandas as pd
import numpy as np
from glob import glob
import nibabel as nib
from nilearn.plotting import plot_img, plot_stat_map, view_img, plot_prob_atlas
from nilearn.regions import connected_label_regions
from nilearn.glm.first_level.hemodynamic_models import spm_hrf
from nilearn.image import concat_imgs, mean_img, index_img, smooth_img
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm import threshold_stats_img
import matplotlib.pyplot as plt
from nilearn.plotting import plot_roi
from nilearn.maskers import NiftiMapsMasker, NiftiSpheresMasker
from scipy.interpolate import interp1d
import seaborn as sns
from scipy.stats.mstats import pearsonr
import scipy.stats as stats

Okay, as I understand it: 
1) Mean center valence and arousal scores. 
2) To obtain positive and negative arousal scores use 
- Pos_arousal -> (arousal_i - valence_i)/sqrt(2)
- Neg_arousal -> (arousal_i - valence_i)/sqrt(2)

- Split like and no-like horror movies. 


- We get best results for MPFC, which is the time course with most structure, try different parameters with NAcc to see if it helps. 
- Use different mask with different parameters for cortical and sub-cortical regions. 

- operationalize peak as delta (x_1 - x_2)/(x_1 + x_2)
- Also, calculate slope between tr_1 and tr_2 that crosses a 1 std. 
- Get the highest correlation for comedy, see when TR happens. 

- What to do if there's more than one peak (i.e., if there's more than two points higher than std).

# Homemade functions

In [68]:
def Merge(dict1, dict2):

    # create a new dictionary by merging the items of the two dictionaries using the union operator (|)
    merged_dict = dict(dict1.items() | dict2.items())
    
    # return the merged dictionary
    return merged_dict

def merge_dictionaries(dict1, dict2):
    merged_dict = dict1.copy()
    merged_dict.update(dict2)
    return merged_dict

In [69]:
# Functions to get different peaks (slope and deltas).
def get_peak_slope(timecourse, timecourse_z, z_threshold, max_min):

    # Store all possible peak in timecourse array.
    possible_slope_peaks = []

    # Loop through each point in timecourse array, except the first one.
    for id_tr in range(1, 15, 1):

        # For positive peaks. 
        if(max_min == "max"):

            # If score is bigger than "z_threshold" z-score in timecourse_z array, append value.
            if(timecourse_z[id_tr] > z_threshold): 

                slope = timecourse[id_tr] - timecourse[id_tr - 1] 
                # Check the slope is not equal to zero. 
                if(slope != 0):
                    possible_slope_peaks.append(slope)
        
        # For negative peaks.
        if(max_min == "min"):

            # If score is bigger than "z_threshold" z-score in timecourse_z array, append value.
            if(timecourse_z[id_tr] < z_threshold): 

                slope = timecourse[id_tr] - timecourse[id_tr - 1] 
                # Check the slope is not equal to zero. 
                if(slope != 0):
                    possible_slope_peaks.append(slope)
    
    average_slope = np.mean(possible_slope_peaks)

    # Return only the first value and average slope.
    if(len(possible_slope_peaks) == 0):
        return 0, 0
    else: 
        return possible_slope_peaks[0], average_slope

def get_narrow_peak_slope(timecourse, z_threshold, max_min):

    # Z-score the 15 data points.
    z_scored_data = stats.zscore(timecourse)

    # Store all possible peak in timecourse array.
    possible_slope_peaks = []

    # Loop through each point in timecourse array, except the first one.
    for id_tr in range(1, 15, 1):

        # For positive peaks.
        if(max_min == "max"):
            # If score is bigger than "z_threshold" z-score in timecourse_z array, append value.
            if(z_scored_data[id_tr] > z_threshold): 

                slope = timecourse[id_tr] - timecourse[id_tr - 1] 
                # Check the slope is not equal to zero. 
                if(slope != 0):
                    possible_slope_peaks.append(slope)
    
        # For negative peaks.
        if(max_min == "min"):
            # If score is bigger than "z_threshold" z-score in timecourse_z array, append value.
            if(z_scored_data[id_tr] < z_threshold): 

                slope = timecourse[id_tr] - timecourse[id_tr - 1] 
                # Check the slope is not equal to zero. 
                if(slope != 0):
                    possible_slope_peaks.append(slope)
    
    average_slope = np.mean(possible_slope_peaks)

    # Return only the first value and average slope.
    if(len(possible_slope_peaks) == 0):
        return 0, 0
    else: 
        return possible_slope_peaks[0], average_slope
    
def get_peak_delta(timecourse, timecourse_z, z_threshold, max_min):

    # Store all possible peak in timecourse array.
    possible_delta_peaks = []

    # Loop through each point in timecourse array, except the first one.
    for id_tr in range(1, 15, 1):

        # For positive peaks. 
        if(max_min == "max"):

            # If score is bigger than "z_threshold" z-score in timecourse_z array, append value.
            if(timecourse_z[id_tr] > z_threshold): 

                delta = (timecourse[id_tr] - timecourse[id_tr - 1])/(timecourse[id_tr]*100) 
                #delta = (timecourse[id_tr] - timecourse[id_tr - 1])/(timecourse[id_tr] + timecourse[id_tr - 1]) 
                # Check the slope is not equal to zero. 
                if(delta != 0):
                    possible_delta_peaks.append(delta)
        
        # For negative peaks.
        if(max_min == "min"):

            # If score is bigger than "z_threshold" z-score in timecourse_z array, append value.
            if(timecourse_z[id_tr] < z_threshold): 

                delta = (timecourse[id_tr] - timecourse[id_tr - 1])/(timecourse[id_tr]*100) 
                #delta = (timecourse[id_tr] - timecourse[id_tr - 1])/(timecourse[id_tr] + timecourse[id_tr - 1]) 
                # Check the slope is not equal to zero. 
                if(delta != 0):
                    possible_delta_peaks.append(delta)
    
    average_delta = np.mean(possible_delta_peaks)

    # Return only the first value and average slope.
    if(len(possible_delta_peaks) == 0):
        return 0, 0
    else: 
        return possible_delta_peaks[0], average_delta
    

def get_narrow_peak_delta(timecourse, z_threshold, max_min):

    # Z-score the 15 data points.
    z_scored_data = stats.zscore(timecourse)

    # Store all possible peak in timecourse array.
    possible_delta_peaks = []

    # Loop through each point in timecourse array, except the first one.
    for id_tr in range(1, 15, 1):

        # For positive peaks.
        if(max_min == "max"):
            # If score is bigger than "z_threshold" z-score in timecourse_z array, append value.
            if(z_scored_data[id_tr] > z_threshold): 

                delta = (timecourse[id_tr] - timecourse[id_tr - 1])/(timecourse[id_tr]*100) 
                #delta = (timecourse[id_tr] - timecourse[id_tr - 1])/(timecourse[id_tr] + timecourse[id_tr - 1]) 
                # Check the slope is not equal to zero. 
                if(delta != 0):
                    possible_delta_peaks.append(delta)
    
        # For negative peaks.
        if(max_min == "min"):
            # If score is bigger than "z_threshold" z-score in timecourse_z array, append value.
            if(z_scored_data[id_tr] < z_threshold): 

                delta = (timecourse[id_tr] - timecourse[id_tr - 1])/(timecourse[id_tr]*100) 
                #delta = (timecourse[id_tr] - timecourse[id_tr - 1])/(timecourse[id_tr] + timecourse[id_tr - 1]) 
                # Check the slope is not equal to zero. 
                if(delta != 0):
                    possible_delta_peaks.append(delta)
    
    average_delta = np.mean(possible_delta_peaks)

    # Return only the first value and average slope.
    if(len(possible_delta_peaks) == 0):
        return 0, 0
    else: 
        return possible_delta_peaks[0], average_delta
     


In [70]:
def plot_trailer_correlations(ROI_df, Trailer_id, ROI_id, TR_corr_dict, type_of_corr, axs):

    if(type_of_corr == "W_score"):
        color="green"
        bonus_y = 0.00
        score_label = "_w"
        text_label = "W corr"
    elif(type_of_corr == "Pos_arousal"):
        color="blue"
        bonus_y = 0.05
        score_label = "_pa"
        text_label = "PA corr"
    elif(type_of_corr == "Neg_arousal"):
        color="red"
        bonus_y = -0.05
        score_label = "_na"
        text_label = "NA corr"

    if(ROI_id == "Bilateral_NAcc"):
        y_coord_seg = 0.3 + bonus_y
        y_coord_whole = 0.4
    elif(ROI_id == "Bilateral_AIns"):
        y_coord_seg = 0.4 + bonus_y
        y_coord_whole = 0.5
    elif(ROI_id == "Bilateral_MPFC"):
        y_coord_seg = 0.4 + bonus_y
        y_coord_whole = 0.5

    # Get the dataframe for the current trailer.
    current_traile_df = ROI_df[Trailer_id]

    # Loop for plotting each significant correlation across the 15 TRs.
    for TR_id in range(15):
        TR_label = str("TR_" + str(TR_id) + score_label) 
        y_coord = current_traile_df[(current_traile_df["ROI"] == ROI_id) & (current_traile_df["TR"] == TR_id)]["Signal"].mean() + bonus_y
        x_coord = TR_id
        if(TR_corr_dict[Trailer_id][TR_label][1] < 0.05):
            if(TR_corr_dict[Trailer_id][TR_label][0] > 0):
                axs.text(x_coord, y_coord, "+*", fontsize=13, color=color, weight="bold", label=text_label)
            else:
                axs.text(x_coord, y_coord, "-*", fontsize=13, color=color, weight="bold", label=text_label)
    
    # Plot the onset correlation if significant.
    #current_traile_df = ROI_df[Trailer_id]
    Onset_label = str("Onset" + score_label)
    #y_coord_seg = (current_traile_df[(current_traile_df["ROI"] == ROI_id)]["Signal"].mean() ) + bonus_y #* 0.7
    x_coord_onset = 3

    if(TR_corr_dict[Trailer_id][Onset_label][1] < 0.05):
        if(TR_corr_dict[Trailer_id][Onset_label][0] > 0):
            axs.text(x_coord_onset, y_coord_seg, "on(+)", fontsize=13, color=color, weight="bold", label=text_label)
        else:
            axs.text(x_coord_onset, y_coord_seg, "on(-)", fontsize=13, color=color, weight="bold", label=text_label)

    # Plot the middle correlation if significant.
    Middle_label = str("Middle" + score_label)
    x_coord_middle = 7
    if(TR_corr_dict[Trailer_id][Middle_label][1] < 0.05):
        if(TR_corr_dict[Trailer_id][Middle_label][0] > 0):
            axs.text(x_coord_middle, y_coord_seg, "mid(+)", fontsize=13, color=color, weight="bold", label=text_label)
        else:
            axs.text(x_coord_middle, y_coord_seg, "mid(-)", fontsize=13, color=color, weight="bold", label=text_label)

    # Plot the offset correlation if significant.
    Offset_label = str("Offset" + score_label)
    x_coord_offset = 13
    if(TR_corr_dict[Trailer_id][Offset_label][1] < 0.05):
        if(TR_corr_dict[Trailer_id][Offset_label][0] > 0):
            axs.text(x_coord_offset, y_coord_seg, "off(+)", fontsize=13, color=color, weight="bold", label=text_label)
        else:
            axs.text(x_coord_offset, y_coord_seg, "off(-)", fontsize=13, color=color, weight="bold", label=text_label)

    # Plot the whole correlation if significant.
    Whole_label = str("Whole" + score_label)
    x_coord_whole = 7
    #y_coord_whole = (current_traile_df[(current_traile_df["ROI"] == ROI_id)]["Signal"].max() ) + bonus_y # * 0.8
    if(TR_corr_dict[Trailer_id][Whole_label][1] < 0.05):
        if(TR_corr_dict[Trailer_id][Whole_label][0] > 0):
            axs.text(x_coord_whole, y_coord_whole, "whole(+)", fontsize=13, color=color, weight="bold", label=text_label)
        else:
            axs.text(x_coord_whole, y_coord_whole, "whole(-)", fontsize=13, color=color, weight="bold", label=text_label)
    



In [71]:
# Define a function to invert scores if needed. 
def transform_scores(score, invert):
    if(invert==1): #if(not invert):
        if(score == 1):  
            return 4
        elif(score == 2):  
            return 3
        elif(score == 3):  
            return 2
        elif(score == 4):  
            return 1
        elif(score == "1"):  
            return 4
        elif(score == "2"):  
            return 3
        elif(score == "3"):  
            return 2
        elif(score == "4"):  
            return 1
        elif(score == "None"):
            return 0
        else: 
            return "Something went wrong here!"
    else:
        if(score == "1"):
            return 1
        elif(score == "2"):
            return 2
        elif(score == "3"):
            return 3
        elif(score == "4"):
            return 4
        elif(score == 1):
            return 1
        elif(score == 2):
            return 2
        elif(score == 3):
            return 3
        elif(score == 4):
            return 4
        elif(score == "None"):
            return 0
        else:
            return "Something went wrong here!"

In [72]:
# Percent signal change .
def get_psc(timecourse):

   # Get number of ROIs and data points in timecourse.
   roi_num = timecourse.shape[1]
   data_length = timecourse.shape[0]

   # Copy timecourse into new array.
   psc_timecourse = np.zeros(timecourse.shape)

   # Warning for empty arrays. 
   if(roi_num ==0):
      print("Watch out, this array is empty!")

   # Loop through every ROI and derive the psc. 
   for id in range(roi_num):

      current_roi_avg = np.mean(timecourse[:, id], axis=0)

      for idx in range(data_length):

         # Formula to get percent signal change -> ((point-avg)/avg)*100.
         psc_timecourse[idx, id] = ((timecourse[idx, id] - current_roi_avg)/ current_roi_avg)*100

   return psc_timecourse

In [73]:
def process_events_data(run_dataframe):

    proccesed_events_df = pd.DataFrame(columns={"Trailer", "Type", "Onset", "Duration", "Offset", "W_score", "A_score", "F_score"}) 

    # Initial fixation 12 sec (TR=6).
    in_fix = 12

    # Time it take subjects to complete questionnaire 12 sec (TR=6). 
    questionnaire_duration = 12

    # All trailers last 30 sec (TR=15). 
    trailer_duration = 30

    # Initialize this variable, though it will change through each iteration of the loop.
    trailer_onset = 0

    # Run a for loop for each row in the df. 
    for id in range(run_dataframe.shape[0]):

        # Get trailer label and separate them accroding to their type. 
        trailer_name = run_dataframe["label"][id]
        trailer_type = "Horror" if "h" in run_dataframe["label"][id] else "Comedy"

        # This onsets don't work, so I need to re-calculate them
        traile_iti = run_dataframe["trial_ITI"][id]
        
        # For the first run add the initial fixation time to the calculation of the first trailer onset. 
        # After the first run, calculate onset by adding previous traile onset, questionnaire duration, and trial iti.
        if (id == 0):
            trailer_onset += in_fix
        else:
            trailer_onset += trailer_duration + questionnaire_duration + traile_iti

        # Calculate trailer onset. 
        trailer_offset = trailer_onset + 30 

        """ 
        For the questionnaire scores, as I understood it. If they were not inverted (["scale_flip"] == 0), then 
        the lower the score the stronger the response. If they were inverted (["scale_flip"] == 1), the higher the 
        score the stronger the response.
        """
        # Check if scaled was flipped and put scores on the same scale. 
        # For me, the most intuitive is that the higher the score, the stronger the response. 
        trailer_watch_score = transform_scores(run_dataframe["exp_Watch.keys"][id], run_dataframe["scale_flip"][id])
        trailer_arousal_score = transform_scores(run_dataframe["exp_Arousal.keys"][id], run_dataframe["scale_flip"][id])
        trailer_feel_score = transform_scores(run_dataframe["exp_Feel.keys"][id], run_dataframe["scale_flip"][id])

        # Place processed data on list, add list to new dataframe, and concat to main dataframe. 
        current_row_data = [[trailer_name, trailer_type, trailer_onset, trailer_duration, trailer_offset, trailer_watch_score, trailer_arousal_score, trailer_feel_score]]
        current_row = pd.DataFrame(data=current_row_data, columns=["Trailer", "Type", "Onset", "Duration", "Offset", "W_score", "A_score", "F_score"]) 
        proccesed_events_df = pd.concat([proccesed_events_df, current_row], ignore_index=True)
        proccesed_events_df = proccesed_events_df[["Trailer", "Type", "Onset", "Offset", "Duration", "W_score", "A_score", "F_score"]]

    # Add a column for the mean centered arousal and valence scores.
    mean_centered_arousal = proccesed_events_df["A_score"]/proccesed_events_df["A_score"].mean()
    mean_centered_valence = proccesed_events_df["W_score"]/proccesed_events_df["W_score"].mean()

    # Derive the positive and negative arousal scores.
    proccesed_events_df["Pos_arousal"] = (mean_centered_arousal+mean_centered_valence)/np.sqrt(2)
    proccesed_events_df["Neg_arousal"] = (mean_centered_arousal-mean_centered_valence)/np.sqrt(2)

    # Derive the positive and negative arousal scaled scores. 
    pos_arousal_mean = proccesed_events_df["Pos_arousal"].mean()
    neg_arousal_mean = proccesed_events_df["Neg_arousal"].mean()

    pos_arousal_std = np.std(proccesed_events_df["Pos_arousal"], axis=0, ddof=1)
    neg_arousal_std = np.std(proccesed_events_df["Neg_arousal"], axis=0, ddof=1)

    proccesed_events_df["Pos_arousal_scaled"] = (proccesed_events_df["Pos_arousal"] - pos_arousal_mean)
    proccesed_events_df["Neg_arousal_scaled"] = (proccesed_events_df["Neg_arousal"] - neg_arousal_mean)

    # Derive scales Watch scores. 
    watch_score_mean = proccesed_events_df["W_score"].mean()    
    watch_score_std = np.std(proccesed_events_df["W_score"], axis=0, ddof=1)
    proccesed_events_df["W_score_scaled"] = (proccesed_events_df["W_score"] - watch_score_mean)

    return proccesed_events_df

In [74]:
def access_trailer(dictionary, trailer_key): 
    try:
        trailer_data = dictionary[trailer_key]
        return trailer_data
    except KeyError:
        pass    

def access_timecourse(dictionary, roi_key):
    
    dictionary_keys = list(dictionary.keys())

    if((roi_key in dictionary_keys) and (dictionary[roi_key] is not None) and (dictionary[roi_key] is not "None")):
        roi_data = np.array(dictionary[roi_key])
        return roi_data
    
    
    # try:
    #     print(dictionary)
    #     if(dictionary[roi_key] == None):
    #         print("This is empty!")
    #         print(dictionary[roi_key])
    #     if(dictionary[roi_key] is not None):
    #         roi_data = np.array(dictionary[roi_key])
    #         return roi_data
    
    # except KeyError:
    #     pass


def trim_timecourse_per_roi(timecourses_dictionary_list, timecourses_dictionary_raw, ROI):

    # 1) Create a new dictionary to store the trimmed timecourses.
    # First file will store the avg timecourse for each trailer per roi (ROI x trailer).
    # Second file will store the timecourse of all subjects for each trailer per roi (ROI x subjects x trailer).
    # Third file will store the timecourse of all subjects for each trailer per roi in a dataframe (ROI x subjects x trailer).
    all_subjects_avg_ROI_timecourse = {}
    all_subjects_ROI_timecourse = {}
    all_subjects_ROI_timecourse_df = {}

    bROI = "Bilateral_" + ROI
    bROI_r1 = "Bilateral_" + ROI + "_r1"
    bROI_r2 = "Bilateral_" + ROI + "_r2"
    rROI = "Right_" + ROI
    rROI_r1 = "Right_" + ROI + "_r1"
    rROI_r2 = "Right_" + ROI + "_r2"
    lROI = "Left_" + ROI
    lROI_r1 = "Left_" + ROI + "_r1"
    lROI_r2 = "Left_" + ROI + "_r2"

    # Get all keys from the first participant dictionary.
    trailer_list = list(timecourses_dictionary_list["sub-01"].keys())
    subjects_list = list(timecourses_dictionary_raw.keys())

    if(ROI != "V1"):
        # Loop through all trailer keys.
        for id in range(len(trailer_list)): 
    
            # Create new participant list with only participants that have the current trailer.
            participants_with_trailer = [d for d in timecourses_dictionary_list if trailer_list[id] in timecourses_dictionary_list[d]] # Store participant id if they have the current trailer.
        
            # Make copy of dictionary with only participants that have the current trailer.
            timecourses_dictionary_list_current_trailer = {k: timecourses_dictionary_list[k] for k in participants_with_trailer}
        
            # Loop through all participants and get the values for the current trailer.
            # Note, a wrapper function just to return the values for the current trailer.
            # if they exist, otherwise return None.
            values = [access_trailer(timecourses_dictionary_list_current_trailer[d], trailer_list[id]) for d in timecourses_dictionary_list_current_trailer] 
        
            values_Bilateral_ROI = [values[d][bROI] for d in range(len(values))]
            values_Right_ROI = [values[d][rROI] for d in range(len(values))]
            values_Left_ROI = [values[d][lROI] for d in range(len(values))]

            # Compute the average for each roi.
            averageBilateral_ROI = np.mean(values_Bilateral_ROI, axis=0)
            averageRight_ROI = np.mean(values_Right_ROI, axis=0)
            averageLeft_ROI = np.mean(values_Left_ROI, axis=0)

            # Row are timepoints, columns are participants (15 x P).
            values_Bilateral_ROI_array = np.array(values_Bilateral_ROI).T 
            values_Right_ROI_array = np.array(values_Right_ROI).T
            values_Left_ROI_array = np.array(values_Left_ROI).T

            # Create columns names for the dataframe.
            TRs = np.arange(0, 15, 1)

            # Creat empty dataframe for current trailer. 
            trailer_df = pd.DataFrame(columns=["Participant", "ROI", "Signal", "TR"])   

            current_participant_dic = {}

            for current_participant in range(len(participants_with_trailer)):

                # Create dictionary with all the timecourses for all trailers for each subject.
                current_participant_dic[participants_with_trailer[current_participant]] = {bROI: values_Bilateral_ROI_array[:, current_participant],
                                                                                       lROI: values_Left_ROI_array[:, current_participant], 
                                                                                       rROI: values_Right_ROI_array[:, current_participant]}
 
                participant_col = np.repeat(str(participants_with_trailer[current_participant]), 15)
                label_bi = np.repeat(bROI, 15)
                label_left = np.repeat(lROI, 15)
                label_right = np.repeat(rROI, 15)
                roi_labels = [label_bi, label_left, label_right]

                # Store current participant values. 
                current_participant_bi = values_Bilateral_ROI_array[:, current_participant]
                current_participant_left = values_Left_ROI_array[:, current_participant]
                current_participant_right = values_Right_ROI_array[:, current_participant]
                current_participant_values = [current_participant_bi, current_participant_left, current_participant_right]

                for x in range(3):

                    current_participant_roi_data = {
                        "Participant": participant_col,
                        "ROI": roi_labels[x],
                        "Signal": current_participant_values[x],
                        "TR": TRs
                    }
            
                    participant_df = pd.DataFrame(data=current_participant_roi_data)

                    trailer_df = pd.concat([trailer_df, participant_df], ignore_index=True)
        
            # store the all in the results dictionaries.
            all_subjects_avg_ROI_timecourse[trailer_list[id]] = {bROI: averageBilateral_ROI, lROI: averageLeft_ROI, rROI: averageRight_ROI}
            all_subjects_ROI_timecourse[trailer_list[id]] = current_participant_dic
            all_subjects_ROI_timecourse_df[trailer_list[id]] = trailer_df
    
    # Get average timecourse for all subjects in the current ROI. 
    Bilateral_ROI_raw_r1_list = [access_timecourse(timecourses_dictionary_raw[sub_id], bROI_r1) for sub_id in subjects_list] # if timecourses_dictionary_raw[sub_id][bROI_r1] is not None
    Bilateral_ROI_raw_r2_list = [access_timecourse(timecourses_dictionary_raw[sub_id], bROI_r2) for sub_id in subjects_list]
    Right_ROI_raw_r1_list = [access_timecourse(timecourses_dictionary_raw[sub_id], rROI_r1) for sub_id in subjects_list]
    Right_ROI_raw_r2_list = [access_timecourse(timecourses_dictionary_raw[sub_id], rROI_r2) for sub_id in subjects_list]
    Left_ROI_raw_r1_list = [access_timecourse(timecourses_dictionary_raw[sub_id], lROI_r1) for sub_id in subjects_list]
    Left_ROI_raw_r2_list = [access_timecourse(timecourses_dictionary_raw[sub_id], lROI_r2) for sub_id in subjects_list]    

    # Remove None values from the list.
    Bilateral_ROI_raw_r1_list = [x for x in Bilateral_ROI_raw_r1_list if x is not None]
    Bilateral_ROI_raw_r2_list = [x for x in Bilateral_ROI_raw_r2_list if x is not None]
    Right_ROI_raw_r1_list = [x for x in Right_ROI_raw_r1_list if x is not None]
    Right_ROI_raw_r2_list = [x for x in Right_ROI_raw_r2_list if x is not None]
    Left_ROI_raw_r1_list = [x for x in Left_ROI_raw_r1_list if x is not None]
    Left_ROI_raw_r2_list = [x for x in Left_ROI_raw_r2_list if x is not None]

    # Get the average timecourse for all subjects in the current ROI.
    Bilateral_ROI_raw_r1_average = np.mean(Bilateral_ROI_raw_r1_list, axis=0)
    Bilateral_ROI_raw_r2_average = np.mean(Bilateral_ROI_raw_r2_list, axis=0)
    Right_ROI_raw_r1_average = np.mean(Right_ROI_raw_r1_list, axis=0)
    Right_ROI_raw_r2_average = np.mean(Right_ROI_raw_r2_list, axis=0)
    Left_ROI_raw_r1_average = np.mean(Left_ROI_raw_r1_list, axis=0)
    Left_ROI_raw_r2_average = np.mean(Left_ROI_raw_r2_list, axis=0)

    # Create TR array.
    TRs_raw = np.arange(0, 378, 1)

    # Put everything in a list 
    ROI_raw_list = [Bilateral_ROI_raw_r1_average, Bilateral_ROI_raw_r2_average, Right_ROI_raw_r1_average, Right_ROI_raw_r2_average, Left_ROI_raw_r1_average, Left_ROI_raw_r2_average]
    ROI_raw_labels = [bROI_r1, bROI_r2, rROI_r1, rROI_r2, lROI_r1, lROI_r2]

    # Creat empty dataframe. 
    all_subjects_avg_ROI_timecourse_raw_df = pd.DataFrame(columns=["ROI", "Signal", "TR"])  

    for ROI_id in range(6):

        current_roi_data = {        
            "ROI": ROI_raw_labels[ROI_id],
            "Signal": ROI_raw_list[ROI_id],
            "TR": TRs_raw}
            
        current_roi_df = pd.DataFrame(data=current_roi_data)

        all_subjects_avg_ROI_timecourse_raw_df = pd.concat([all_subjects_avg_ROI_timecourse_raw_df, current_roi_df], ignore_index=True)


    return all_subjects_avg_ROI_timecourse, all_subjects_ROI_timecourse, all_subjects_ROI_timecourse_df, all_subjects_avg_ROI_timecourse_raw_df


<>:12: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:12: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/var/folders/0z/1yt2h6410kb7_mgghf4q28z00000gn/T/ipykernel_3366/1498170635.py:12: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if((roi_key in dictionary_keys) and (dictionary[roi_key] is not None) and (dictionary[roi_key] is not "None")):


In [75]:
# Open a datasets directory. 
fd = os.open("/Users/luisalvarez/Documents/Datasets", os.O_RDONLY)

# Use os.fchdir() method to change the current dir/folder.
os.fchdir(fd)

# Safe check- Print current working directory
print("Current working dir : %s" % os.getcwd())

Current working dir : /Users/luisalvarez/Documents/Datasets


In [76]:

def getROIs_timecourse(participant_num, type, sub_motion): 

    # Check if the type is correct.
    if (type == "psc") or (type == "zscore"):
        print("Getting ROI timecourse for participant: " + participant_num + " and type: " + type)
    else:
        raise ValueError("Type not recognized. Please use 'psc' or 'zscore'.")

    # Add code to flag if something that is not a number is passed. 

    ## 1) Load data.
    # Load relevant files for participant
    sub_run1_func_path = "MovieData_BIDS_preproc/sub-" + participant_num + "/func/sub-" + participant_num + "_task-movie_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"
    sub_run2_func_path = "MovieData_BIDS_preproc/sub-" + participant_num + "/func/sub-" + participant_num + "_task-movie_run-02_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"

    sub_run1_mask_path = "MovieData_BIDS_preproc/sub-" + participant_num + "/func/sub-" + participant_num + "_task-movie_run-01_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz"
    sub_run2_mask_path = "MovieData_BIDS_preproc/sub-" + participant_num + "/func/sub-" + participant_num + "_task-movie_run-02_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz"

    sub_run1_events_path = "MovieData_BIDS_raw/sub-" + participant_num + "/func/sub-" + participant_num + "_task-movie_run-01_events.csv"
    sub_run2_events_path = "MovieData_BIDS_raw/sub-" + participant_num + "/func/sub-" + participant_num + "_task-movie_run-02_events.csv"

    sub_run1_confounds_path = "MovieData_BIDS_preproc/sub-" + participant_num + "/func/sub-" + participant_num + "_task-movie_run-01_desc-confounds_timeseries.tsv"
    sub_run2_confounds_path = "MovieData_BIDS_preproc/sub-" + participant_num + "/func/sub-" + participant_num + "_task-movie_run-02_desc-confounds_timeseries.tsv"

    # Calculate relevant parameters for GLM and ROI time-course analysis.
    func_file = nib.load(sub_run1_func_path)
    func_data = func_file.get_fdata()
    n_vols = func_data.shape[3]
    TR = 2
    n_timepoints = n_vols*TR

    # Load raw events files. 
    sub_run1_events_df = pd.read_csv(sub_run1_events_path, index_col=0)
    sub_run2_events_df = pd.read_csv(sub_run2_events_path, index_col=0)

    ## 2) Process files. 
    # Process event files. 
    sub_run1_p_events = process_events_data(sub_run1_events_df)
    sub_run2_p_events = process_events_data(sub_run2_events_df)

    # Down-sample time onsets to get vol onsets. 
    # Create array from 0 to 'n_timepoints' in steps of 1.
    time_scale = np.arange(0, n_timepoints, 1)  

    # Create array from 0 to 'n_timepoints' in steps of 2.
    vol_scale = np.arange(0, n_timepoints, TR)  

    # Get the labels of each trailer for each run. 
    run1_trailer_labels = sub_run1_p_events["Trailer"].tolist()
    run2_trailer_labels = sub_run2_p_events["Trailer"].tolist()

    # Create dictionary variable to store arrays with onset values for each trailer. 
    run1_onsets = {}
    run2_onsets = {}

    # Create a dictionary with all the onsets for each trailer in each run. 
    for id in range(len(run1_trailer_labels)):

        # Create array of zeros.
        run1_trailer_onsets = np.zeros(n_timepoints)
        run2_trailer_onsets = np.zeros(n_timepoints)

        # Get onset time. 
        run1_current_trailer_onset = sub_run1_p_events["Onset"][id]
        run2_current_trailer_onset = sub_run2_p_events["Onset"][id]

        # Assign 1 to such onset all the way til the end of the trailer (30 sec) in the array of zeros.
        # Adjust for lag: add 4 seconds at the onset and offset
        # Let's add 4 seconds to the onset and offset to account for the lag in the BOLD signal.
        #run1_trailer_onsets[int(run1_current_trailer_onset + 4):int(run1_current_trailer_onset)+ 30 + 4] = 1
        #run2_trailer_onsets[int(run2_current_trailer_onset + 4):int(run2_current_trailer_onset)+ 30 + 4] = 1
        run1_trailer_onsets[int(run1_current_trailer_onset + 5):int(run1_current_trailer_onset)+ 30 + 5] = 1
        run2_trailer_onsets[int(run2_current_trailer_onset + 5):int(run2_current_trailer_onset)+ 30 + 5] = 1

        # Create resampler objects for each trailer/run of reward.
        run1_resampler = interp1d(time_scale, run1_trailer_onsets)
        run2_resampler = interp1d(time_scale, run2_trailer_onsets)

        # Create downsampled arrays for each trailer. 
        # Note this vol arrays are half the length than the time arrays.
        run1_trailer_vol_onsets = run1_resampler(vol_scale)
        run2_trailer_vol_onsets = run2_resampler(vol_scale)

        # Append/store the downsampled volumes arrays to each dictionary.
        # I'm doing it this way, so the code is more interpretable
        run1_onsets[run1_trailer_labels[id]] = run1_trailer_vol_onsets
        run2_onsets[run2_trailer_labels[id]] = run2_trailer_vol_onsets

    ## 3) Load confound data. 
    sub_run1_confounds_df = pd.read_csv(sub_run1_confounds_path, sep='\t')
    sub_run2_confounds_df = pd.read_csv(sub_run2_confounds_path, sep='\t')
    default_confounds = ["white_matter", "csf", "csf_wm", "framewise_displacement", "dvars", "rmsd", "tcompcor"] 
    # ["white_matter", "csf", "csf_wm", "framewise_displacement",  "rmsd", "tcompcor"]
    
    cohen_confounds = ['c_comp_cor_00','c_comp_cor_01','c_comp_cor_02','w_comp_cor_00','w_comp_cor_01','w_comp_cor_02',
                       'w_comp_cor_03','w_comp_cor_04','trans_x','trans_y','trans_z','rot_x','rot_y','rot_z','trans_x_derivative1'
                       ,'trans_y_derivative1','trans_z_derivative1','rot_x_derivative1','rot_y_derivative1','rot_z_derivative1',
                       'trans_x_power2','trans_y_power2','trans_z_power2','rot_x_power2','rot_y_power2','rot_z_power2',
                       'trans_x_derivative1_power2','trans_y_derivative1_power2','trans_z_derivative1_power2','rot_x_derivative1_power2',
                       'rot_y_derivative1_power2','rot_z_derivative1_power2','cosine00'] #, 'c_comp_cor_03', 'c_comp_cor_04'

    sub_run1_motion_s_confounds = [i for i in sub_run1_confounds_df.columns if "state" in i] #"motion"
    sub_run2_motion_s_confounds = [i for i in sub_run2_confounds_df.columns if "state" in i] 

    sub_run1_motion_confounds = [i for i in sub_run1_confounds_df.columns if "motion" in i] #"motion"
    sub_run2_motion_confounds = [i for i in sub_run2_confounds_df.columns if "motion" in i] 

    sub_run1_motion_rot_confounds = [i for i in sub_run1_confounds_df.columns if "rot" in i] 
    sub_run2_motion_rot_confounds = [i for i in sub_run2_confounds_df.columns if "rot" in i] 

    sub_run1_motion_trans_confounds = [i for i in sub_run1_confounds_df.columns if "trans" in i] 
    sub_run2_motion_trans_confounds = [i for i in sub_run2_confounds_df.columns if "trans" in i] 

    sub_run1_filtered_confounds_df = sub_run1_confounds_df[default_confounds + sub_run1_motion_s_confounds + sub_run1_motion_rot_confounds + sub_run1_motion_trans_confounds]
    sub_run2_filtered_confounds_df = sub_run2_confounds_df[default_confounds + sub_run2_motion_s_confounds + sub_run2_motion_rot_confounds + sub_run2_motion_trans_confounds]

    sub_run1_cohen_confounds_df = sub_run1_confounds_df[cohen_confounds ] #+ sub_run1_motion_s_confounds
    sub_run2_cohen_confounds_df = sub_run2_confounds_df[cohen_confounds ] #+ sub_run2_motion_s_confounds

    # Change NaNs to 0s. 
    sub_run1_filtered_confounds_df = sub_run1_filtered_confounds_df.fillna(0) 
    sub_run2_filtered_confounds_df = sub_run2_filtered_confounds_df.fillna(0) 

    sub_run1_cohen_confounds_df = sub_run1_cohen_confounds_df.fillna(0) 
    sub_run2_cohen_confounds_df = sub_run2_cohen_confounds_df.fillna(0) 

    # Specify the parameters to apply to the given analysis.
    if(type=="zscore"):
        detrend = True
        standardize = type
        standardize_confounds = False
        confounds1 = sub_run1_cohen_confounds_df
        confounds2 = sub_run2_cohen_confounds_df
        smoothing=8
        set_of_seeds = [(10, 12, -2), (-10, 12, -2), # Nucleus accumbes (NAcc) right, left
                (34, 24, -4), (-34, 24, -4), # Anterior Insula right (AIns), left
                (4, 45, 0), (-4, 45, 0)] # Medial Prefrontal Cortex (MPFC) right, left

    # Let's try to detrend and not standarize confounds for PSC ... 
    elif(type=="psc"):
        detrend = False #True # False
        standardize = type
        standardize_confounds = type #False #type
        confounds1 = sub_run1_filtered_confounds_df
        confounds2 = sub_run2_filtered_confounds_df
        smoothing= 6 # from 4 to 6
        set_of_seeds = [(10, 12, -2), (-10, 12, -2), # Nucleus accumbes (NAcc) right, left
                (34, 24, -4), (-34, 24, -4), # Anterior Insula right (AIns), left
                (4, 45, 0), (-4, 45, 0), # Medial Prefrontal Cortex (MPFC) right, left
                (8, -76, 10), (-8, -76, 10)] # Visual cortex right, left

    # Apply TPJ. 
    ## 4) Apply mask to func data. 
    masker_AIM_ROI_r1 = NiftiSpheresMasker(
        seeds=set_of_seeds,
        allow_overlap=True,
        smoothing_fwhm=smoothing, # Applying a Gaussian filter with a 4mm kernel
        detrend=detrend,
        radius=6, 
        mask_img=sub_run1_mask_path,
        standardize=standardize, 
        t_r=2,
        standardize_confounds=standardize_confounds,
        high_pass=1/360, # High cutoff frequency in Hertz.
        #low_pass=0.1 # from 1.0
        )

    masker_AIM_ROI_r2 = NiftiSpheresMasker(
        seeds=set_of_seeds,
        allow_overlap=True,
        smoothing_fwhm=smoothing, # Applying a Gaussian filter with a 4mm kernel
        detrend=detrend,
        radius=6, 
        mask_img=sub_run2_mask_path,
        standardize=standardize, 
        t_r=2,
        standardize_confounds=standardize_confounds,
        high_pass=1/360, # High cutoff frequency in Hertz.
        #low_pass=0.1 # from 1.0
        )

    ROI_raw_timecourses = {}
    # Mask the func data and get a time series for the ROI. 
    # Note this is similar to fitting the GLM, but without the event files.
    ## Add if statement to only apply the masker if the run has acceptable motion parameters. 

    if((sub_motion[0] < 1) and (sub_motion[1] < 1)):

        # Apply function to get the percent signal change from each ROI timecourse. 
        sub_r1_AIM_ROI = masker_AIM_ROI_r1.fit_transform(sub_run1_func_path, confounds=confounds1)

        ## 5) Get the timecourses from each movie trailer. 
        # Create dictionary variable to store arrays with time series arrays for each trailer.
        run1_timeseries = {}

        # Get the trailers presented in each run. 
        r1_keys = list(run1_onsets.keys())

        # Loop through each traile and get its corresponding ROI timecourse
        for id in range(len(r1_keys)):

            run1_timeseries[r1_keys[id]] = {
                "Bilateral_NAcc": np.mean(sub_r1_AIM_ROI[run1_onsets[r1_keys[id]].astype(bool)][:, 0:2], axis=1),
                "Left_NAcc": sub_r1_AIM_ROI[run1_onsets[r1_keys[id]].astype(bool)][:, 1],
                "Right_NAcc": sub_r1_AIM_ROI[run1_onsets[r1_keys[id]].astype(bool)][:, 0],
                "Bilateral_AIns": np.mean(sub_r1_AIM_ROI[run1_onsets[r1_keys[id]].astype(bool)][:, 2:4], axis=1),
                "Left_AIns": sub_r1_AIM_ROI[run1_onsets[r1_keys[id]].astype(bool)][:, 3],
                "Right_AIns": sub_r1_AIM_ROI[run1_onsets[r1_keys[id]].astype(bool)][:, 2],
                "Bilateral_MPFC": np.mean(sub_r1_AIM_ROI[run1_onsets[r1_keys[id]].astype(bool)][:, 4:6], axis=1),
                "Left_MPFC": sub_r1_AIM_ROI[run1_onsets[r1_keys[id]].astype(bool)][:, 5],
                "Right_MPFC": sub_r1_AIM_ROI[run1_onsets[r1_keys[id]].astype(bool)][:, 4]}
        
        # This code gives me the un-trimmed timecourses for each ROI. 
        if(type=="psc"):
            ROI_raw_timecourses["Bilateral_NAcc_r1"] = np.mean(sub_r1_AIM_ROI[:, 0:2], axis=1)
            ROI_raw_timecourses["Left_NAcc_r1"] = sub_r1_AIM_ROI[:, 1]
            ROI_raw_timecourses["Right_NAcc_r1"] = sub_r1_AIM_ROI[:, 0]
            ROI_raw_timecourses["Bilateral_AIns_r1"] = np.mean(sub_r1_AIM_ROI[:, 2:4], axis=1)
            ROI_raw_timecourses["Left_AIns_r1"] = sub_r1_AIM_ROI[:, 3]
            ROI_raw_timecourses["Right_AIns_r1"] = sub_r1_AIM_ROI[:, 2]
            ROI_raw_timecourses["Bilateral_MPFC_r1"] = np.mean(sub_r1_AIM_ROI[:, 4:6], axis=1)
            ROI_raw_timecourses["Left_MPFC_r1"] = sub_r1_AIM_ROI[:, 5]
            ROI_raw_timecourses["Right_MPFC_r1"] = sub_r1_AIM_ROI[:, 4]
            ROI_raw_timecourses["Bilateral_V1_r1"] = np.mean(sub_r1_AIM_ROI[:, 6:8], axis=1)
            ROI_raw_timecourses["Left_V1_r1"] = sub_r1_AIM_ROI[:, 6]
            ROI_raw_timecourses["Right_V1_r1"] = sub_r1_AIM_ROI[:, 7]
        else:
            ROI_raw_timecourses["Bilateral_NAcc_r1"] = np.mean(sub_r1_AIM_ROI[:, 0:2], axis=1)
            ROI_raw_timecourses["Left_NAcc_r1"] = sub_r1_AIM_ROI[:, 1]
            ROI_raw_timecourses["Right_NAcc_r1"] = sub_r1_AIM_ROI[:, 0]
            ROI_raw_timecourses["Bilateral_AIns_r1"] = np.mean(sub_r1_AIM_ROI[:, 2:4], axis=1)
            ROI_raw_timecourses["Left_AIns_r1"] = sub_r1_AIM_ROI[:, 3]
            ROI_raw_timecourses["Right_AIns_r1"] = sub_r1_AIM_ROI[:, 2]
            ROI_raw_timecourses["Bilateral_MPFC_r1"] = np.mean(sub_r1_AIM_ROI[:, 4:6], axis=1)
            ROI_raw_timecourses["Left_MPFC_r1"] = sub_r1_AIM_ROI[:, 5]
            ROI_raw_timecourses["Right_MPFC_r1"] = sub_r1_AIM_ROI[:, 4]



    # Apply for second run.
    if((sub_motion[2] < 1) and (sub_motion[3] < 1)):

        # Apply function to get the percent signal change from each ROI timecourse.
        sub_r2_AIM_ROI = masker_AIM_ROI_r2.fit_transform(sub_run2_func_path, confounds=confounds2)

        # Create dictionary variable to store arrays with time series arrays for each trailer.
        run2_timeseries = {}

        r2_keys = list(run2_onsets.keys())

        # Loop through each traile and get its corresponding ROI timecourse
        for id in range(len(r2_keys)):
            run2_timeseries[r2_keys[id]] = {
                "Bilateral_NAcc": np.mean(sub_r2_AIM_ROI[run2_onsets[r2_keys[id]].astype(bool)][:, 0:2], axis=1),
                "Left_NAcc": sub_r2_AIM_ROI[run2_onsets[r2_keys[id]].astype(bool)][:, 1],
                "Right_NAcc": sub_r2_AIM_ROI[run2_onsets[r2_keys[id]].astype(bool)][:, 0],
                "Bilateral_AIns": np.mean(sub_r2_AIM_ROI[run2_onsets[r2_keys[id]].astype(bool)][:, 2:4], axis=1),
                "Left_AIns": sub_r2_AIM_ROI[run2_onsets[r2_keys[id]].astype(bool)][:, 3],
                "Right_AIns": sub_r2_AIM_ROI[run2_onsets[r2_keys[id]].astype(bool)][:, 2],
                "Bilateral_MPFC": np.mean(sub_r2_AIM_ROI[run2_onsets[r2_keys[id]].astype(bool)][:, 4:6], axis=1),
                "Left_MPFC": sub_r2_AIM_ROI[run2_onsets[r2_keys[id]].astype(bool)][:, 5],
                "Right_MPFC": sub_r2_AIM_ROI[run2_onsets[r2_keys[id]].astype(bool)][:, 4]}
        
        # This code gives me the un-trimmed timecourses for each ROI.
        if(type=="psc"):
            ROI_raw_timecourses["Bilateral_NAcc_r2"] = np.mean(sub_r2_AIM_ROI[:, 0:2], axis=1)
            ROI_raw_timecourses["Left_NAcc_r2"] = sub_r2_AIM_ROI[:, 1]
            ROI_raw_timecourses["Right_NAcc_r2"] = sub_r2_AIM_ROI[:, 0]
            ROI_raw_timecourses["Bilateral_AIns_r2"] = np.mean(sub_r2_AIM_ROI[:, 2:4], axis=1)
            ROI_raw_timecourses["Left_AIns_r2"] = sub_r2_AIM_ROI[:, 3]
            ROI_raw_timecourses["Right_AIns_r2"] = sub_r2_AIM_ROI[:, 2]
            ROI_raw_timecourses["Bilateral_MPFC_r2"] = np.mean(sub_r2_AIM_ROI[:, 4:6], axis=1)
            ROI_raw_timecourses["Left_MPFC_r2"] = sub_r2_AIM_ROI[:, 5]
            ROI_raw_timecourses["Right_MPFC_r2"] = sub_r2_AIM_ROI[:, 4]
            ROI_raw_timecourses["Bilateral_V1_r2"] = np.mean(sub_r2_AIM_ROI[:, 6:8], axis=1)
            ROI_raw_timecourses["Left_V1_r2"] = sub_r2_AIM_ROI[:, 6]
            ROI_raw_timecourses["Right_V1_r2"] = sub_r2_AIM_ROI[:, 7]
        else:
            ROI_raw_timecourses["Bilateral_NAcc_r2"] = np.mean(sub_r2_AIM_ROI[:, 0:2], axis=1)
            ROI_raw_timecourses["Left_NAcc_r2"] = sub_r2_AIM_ROI[:, 1]
            ROI_raw_timecourses["Right_NAcc_r2"] = sub_r2_AIM_ROI[:, 0]
            ROI_raw_timecourses["Bilateral_AIns_r2"] = np.mean(sub_r2_AIM_ROI[:, 2:4], axis=1)
            ROI_raw_timecourses["Left_AIns_r2"] = sub_r2_AIM_ROI[:, 3]
            ROI_raw_timecourses["Right_AIns_r2"] = sub_r2_AIM_ROI[:, 2]
            ROI_raw_timecourses["Bilateral_MPFC_r2"] = np.mean(sub_r2_AIM_ROI[:, 4:6], axis=1)
            ROI_raw_timecourses["Left_MPFC_r2"] = sub_r2_AIM_ROI[:, 5]
            ROI_raw_timecourses["Right_MPFC_r2"] = sub_r2_AIM_ROI[:, 4]

    ## 6) Merge timecourses and events data. 
    if((sub_motion[0] < 1) and (sub_motion[2] < 1)): 
        all_timeseries = merge_dictionaries(run1_timeseries, run2_timeseries)   
        sub_all_p_events = pd.concat([sub_run1_p_events, sub_run2_p_events], ignore_index=True)
        print("Participant " + participant_num + " has acceptable data for both runs.")
    elif((sub_motion[0] < 1) and (sub_motion[2] > 1)): 
        all_timeseries = run1_timeseries
        sub_all_p_events = sub_run1_p_events
        print("Participant " + participant_num + " has acceptable data for run 1 only.")
    elif((sub_motion[0] > 1) and (sub_motion[2] < 1)):
        all_timeseries = run2_timeseries
        sub_all_p_events = sub_run2_p_events
        print("Participant " + participant_num + " has acceptable data for run 2 only.")
    else:
        print("Participant " + participant_num + " has unacceptable data for both runs.")
        all_timeseries = {}
        sub_all_p_events = []
        
    return all_timeseries, sub_all_p_events, ROI_raw_timecourses



In [90]:
def getROIs_timecourse_detrend(participant_num, type, sub_motion): 

    # Check if the type is correct.
    if (type == "psc") or (type == "zscore"):
        print("Getting ROI timecourse for participant: " + participant_num + " and type: " + type)
    else:
        raise ValueError("Type not recognized. Please use 'psc' or 'zscore'.")

    # Add code to flag if something that is not a number is passed. 

    ## 1) Load data.
    # Load relevant files for participant
    sub_run1_func_path = "MovieData_BIDS_preproc/sub-" + participant_num + "/func/sub-" + participant_num + "_task-movie_run-01_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"
    sub_run2_func_path = "MovieData_BIDS_preproc/sub-" + participant_num + "/func/sub-" + participant_num + "_task-movie_run-02_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"

    sub_run1_mask_path = "MovieData_BIDS_preproc/sub-" + participant_num + "/func/sub-" + participant_num + "_task-movie_run-01_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz"
    sub_run2_mask_path = "MovieData_BIDS_preproc/sub-" + participant_num + "/func/sub-" + participant_num + "_task-movie_run-02_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz"

    sub_run1_events_path = "MovieData_BIDS_raw/sub-" + participant_num + "/func/sub-" + participant_num + "_task-movie_run-01_events.csv"
    sub_run2_events_path = "MovieData_BIDS_raw/sub-" + participant_num + "/func/sub-" + participant_num + "_task-movie_run-02_events.csv"

    sub_run1_confounds_path = "MovieData_BIDS_preproc/sub-" + participant_num + "/func/sub-" + participant_num + "_task-movie_run-01_desc-confounds_timeseries.tsv"
    sub_run2_confounds_path = "MovieData_BIDS_preproc/sub-" + participant_num + "/func/sub-" + participant_num + "_task-movie_run-02_desc-confounds_timeseries.tsv"

    # Calculate relevant parameters for GLM and ROI time-course analysis.
    func_file = nib.load(sub_run1_func_path)
    func_data = func_file.get_fdata()
    n_vols = func_data.shape[3]
    TR = 2
    n_timepoints = n_vols*TR

    # Load raw events files. 
    sub_run1_events_df = pd.read_csv(sub_run1_events_path, index_col=0)
    sub_run2_events_df = pd.read_csv(sub_run2_events_path, index_col=0)

    ## 2) Process files. 
    # Process event files. 
    sub_run1_p_events = process_events_data(sub_run1_events_df)
    sub_run2_p_events = process_events_data(sub_run2_events_df)

    # Down-sample time onsets to get vol onsets. 
    # Create array from 0 to 'n_timepoints' in steps of 1.
    time_scale = np.arange(0, n_timepoints, 1)  

    # Create array from 0 to 'n_timepoints' in steps of 2.
    vol_scale = np.arange(0, n_timepoints, TR)  

    # Get the labels of each trailer for each run. 
    run1_trailer_labels = sub_run1_p_events["Trailer"].tolist()
    run2_trailer_labels = sub_run2_p_events["Trailer"].tolist()

    # Create dictionary variable to store arrays with onset values for each trailer. 
    run1_onsets = {}
    run2_onsets = {}

    # Create a dictionary with all the onsets for each trailer in each run. 
    for id in range(len(run1_trailer_labels)):

        # Create array of zeros.
        run1_trailer_onsets = np.zeros(n_timepoints)
        run2_trailer_onsets = np.zeros(n_timepoints)

        # Get onset time. 
        run1_current_trailer_onset = sub_run1_p_events["Onset"][id]
        run2_current_trailer_onset = sub_run2_p_events["Onset"][id]

        # Assign 1 to such onset all the way til the end of the trailer (30 sec) in the array of zeros.
        # Adjust for lag: add 4 seconds at the onset and offset
        # Let's add 4 seconds to the onset and offset to account for the lag in the BOLD signal.
        #run1_trailer_onsets[int(run1_current_trailer_onset + 4):int(run1_current_trailer_onset)+ 30 + 4] = 1
        #run2_trailer_onsets[int(run2_current_trailer_onset + 4):int(run2_current_trailer_onset)+ 30 + 4] = 1
        run1_trailer_onsets[int(run1_current_trailer_onset + 5):int(run1_current_trailer_onset)+ 30 + 5] = 1
        run2_trailer_onsets[int(run2_current_trailer_onset + 5):int(run2_current_trailer_onset)+ 30 + 5] = 1

        # Create resampler objects for each trailer/run of reward.
        run1_resampler = interp1d(time_scale, run1_trailer_onsets)
        run2_resampler = interp1d(time_scale, run2_trailer_onsets)

        # Create downsampled arrays for each trailer. 
        # Note this vol arrays are half the length than the time arrays.
        run1_trailer_vol_onsets = run1_resampler(vol_scale)
        run2_trailer_vol_onsets = run2_resampler(vol_scale)

        # Append/store the downsampled volumes arrays to each dictionary.
        # I'm doing it this way, so the code is more interpretable
        run1_onsets[run1_trailer_labels[id]] = run1_trailer_vol_onsets
        run2_onsets[run2_trailer_labels[id]] = run2_trailer_vol_onsets

    ## 3) Load confound data. 
    sub_run1_confounds_df = pd.read_csv(sub_run1_confounds_path, sep='\t')
    sub_run2_confounds_df = pd.read_csv(sub_run2_confounds_path, sep='\t')
    default_confounds = ["white_matter", "csf", "csf_wm", "framewise_displacement", "dvars", "rmsd", "tcompcor",
                         'c_comp_cor_00','c_comp_cor_01', 'w_comp_cor_00','w_comp_cor_01', 'a_comp_cor_00','a_comp_cor_01']
    # ["white_matter", "csf", "csf_wm", "framewise_displacement",  "rmsd", "tcompcor"]
    
    cohen_confounds = ['c_comp_cor_00','c_comp_cor_01','c_comp_cor_02','w_comp_cor_00','w_comp_cor_01','w_comp_cor_02',
                       'w_comp_cor_03','w_comp_cor_04','trans_x','trans_y','trans_z','rot_x','rot_y','rot_z','trans_x_derivative1'
                       ,'trans_y_derivative1','trans_z_derivative1','rot_x_derivative1','rot_y_derivative1','rot_z_derivative1',
                       'trans_x_power2','trans_y_power2','trans_z_power2','rot_x_power2','rot_y_power2','rot_z_power2',
                       'trans_x_derivative1_power2','trans_y_derivative1_power2','trans_z_derivative1_power2','rot_x_derivative1_power2',
                       'rot_y_derivative1_power2','rot_z_derivative1_power2','cosine00'] #, 'c_comp_cor_03', 'c_comp_cor_04'

    sub_run1_motion_s_confounds = [i for i in sub_run1_confounds_df.columns if "state" in i] #"motion"
    sub_run2_motion_s_confounds = [i for i in sub_run2_confounds_df.columns if "state" in i] 

    sub_run1_motion_confounds = [i for i in sub_run1_confounds_df.columns if "motion" in i] #"motion"
    sub_run2_motion_confounds = [i for i in sub_run2_confounds_df.columns if "motion" in i] 

    sub_run1_motion_rot_confounds = [i for i in sub_run1_confounds_df.columns if "rot" in i] 
    sub_run2_motion_rot_confounds = [i for i in sub_run2_confounds_df.columns if "rot" in i] 

    sub_run1_motion_trans_confounds = [i for i in sub_run1_confounds_df.columns if "trans" in i] 
    sub_run2_motion_trans_confounds = [i for i in sub_run2_confounds_df.columns if "trans" in i] 

    sub_run1_filtered_confounds_df = sub_run1_confounds_df[default_confounds + sub_run1_motion_s_confounds + sub_run1_motion_rot_confounds + sub_run1_motion_trans_confounds]
    sub_run2_filtered_confounds_df = sub_run2_confounds_df[default_confounds + sub_run2_motion_s_confounds + sub_run2_motion_rot_confounds + sub_run2_motion_trans_confounds]

    sub_run1_cohen_confounds_df = sub_run1_confounds_df[cohen_confounds ] #+ sub_run1_motion_s_confounds
    sub_run2_cohen_confounds_df = sub_run2_confounds_df[cohen_confounds ] #+ sub_run2_motion_s_confounds

    # Change NaNs to 0s. 
    sub_run1_filtered_confounds_df = sub_run1_filtered_confounds_df.fillna(0) 
    sub_run2_filtered_confounds_df = sub_run2_filtered_confounds_df.fillna(0) 

    sub_run1_cohen_confounds_df = sub_run1_cohen_confounds_df.fillna(0) 
    sub_run2_cohen_confounds_df = sub_run2_cohen_confounds_df.fillna(0) 

    # Specify the parameters to apply to the given analysis.
    if(type=="zscore"):
        detrend = True
        standardize = type
        standardize_confounds = False
        confounds1 = sub_run1_cohen_confounds_df
        confounds2 = sub_run2_cohen_confounds_df
        smoothing=8
        set_of_seeds = [(10, 12, -2), (-10, 12, -2), # Nucleus accumbes (NAcc) right, left
                (34, 24, -4), (-34, 24, -4), # Anterior Insula right (AIns), left
                (4, 45, 0), (-4, 45, 0)] # Medial Prefrontal Cortex (MPFC) right, left

    # Let's try to detrend and not standarize confounds for PSC ... 
    elif(type=="psc"):
        detrend = True #True # False
        standardize = type
        standardize_confounds = type #False #type
        confounds1 = sub_run1_filtered_confounds_df
        confounds2 = sub_run2_filtered_confounds_df
        smoothing= 6 # from 4 to 6
        set_of_seeds = [(10, 12, -2), (-10, 12, -2), # Nucleus accumbes (NAcc) right, left
                (34, 24, -4), (-34, 24, -4), # Anterior Insula right (AIns), left
                (4, 45, 0), (-4, 45, 0), # Medial Prefrontal Cortex (MPFC) right, left
                (8, -76, 10), (-8, -76, 10)] # Visual cortex right, left

    # Apply TPJ. 
    ## 4) Apply mask to func data. 
    masker_AIM_ROI_r1 = NiftiSpheresMasker(
        seeds=set_of_seeds,
        allow_overlap=True,
        smoothing_fwhm=smoothing, # Applying a Gaussian filter with a 4mm kernel
        detrend=detrend,
        radius=6, 
        mask_img=sub_run1_mask_path,
        standardize=standardize, 
        t_r=2,
        standardize_confounds=standardize_confounds,
        high_pass=1/360, # High cutoff frequency in Hertz.
        #low_pass=0.1 # from 1.0
        )

    masker_AIM_ROI_r2 = NiftiSpheresMasker(
        seeds=set_of_seeds,
        allow_overlap=True,
        smoothing_fwhm=smoothing, # Applying a Gaussian filter with a 4mm kernel
        detrend=detrend,
        radius=6, 
        mask_img=sub_run2_mask_path,
        standardize=standardize, 
        t_r=2,
        standardize_confounds=standardize_confounds,
        high_pass=1/360, # High cutoff frequency in Hertz.
        #low_pass=0.1 # from 1.0
        )

    ROI_raw_timecourses = {}
    # Mask the func data and get a time series for the ROI. 
    # Note this is similar to fitting the GLM, but without the event files.
    ## Add if statement to only apply the masker if the run has acceptable motion parameters. 

    if((sub_motion[0] < 1) and (sub_motion[1] < 1)):

        # Apply function to get the percent signal change from each ROI timecourse. 
        sub_r1_AIM_ROI = masker_AIM_ROI_r1.fit_transform(sub_run1_func_path, confounds=confounds1)

        ## 5) Get the timecourses from each movie trailer. 
        # Create dictionary variable to store arrays with time series arrays for each trailer.
        run1_timeseries = {}

        # Get the trailers presented in each run. 
        r1_keys = list(run1_onsets.keys())

        # Loop through each traile and get its corresponding ROI timecourse
        for id in range(len(r1_keys)):

            run1_timeseries[r1_keys[id]] = {
                "Bilateral_NAcc": np.mean(sub_r1_AIM_ROI[run1_onsets[r1_keys[id]].astype(bool)][:, 0:2], axis=1),
                "Left_NAcc": sub_r1_AIM_ROI[run1_onsets[r1_keys[id]].astype(bool)][:, 1],
                "Right_NAcc": sub_r1_AIM_ROI[run1_onsets[r1_keys[id]].astype(bool)][:, 0],
                "Bilateral_AIns": np.mean(sub_r1_AIM_ROI[run1_onsets[r1_keys[id]].astype(bool)][:, 2:4], axis=1),
                "Left_AIns": sub_r1_AIM_ROI[run1_onsets[r1_keys[id]].astype(bool)][:, 3],
                "Right_AIns": sub_r1_AIM_ROI[run1_onsets[r1_keys[id]].astype(bool)][:, 2],
                "Bilateral_MPFC": np.mean(sub_r1_AIM_ROI[run1_onsets[r1_keys[id]].astype(bool)][:, 4:6], axis=1),
                "Left_MPFC": sub_r1_AIM_ROI[run1_onsets[r1_keys[id]].astype(bool)][:, 5],
                "Right_MPFC": sub_r1_AIM_ROI[run1_onsets[r1_keys[id]].astype(bool)][:, 4]}
        
        # This code gives me the un-trimmed timecourses for each ROI. 
        if(type=="psc"):
            ROI_raw_timecourses["Bilateral_NAcc_r1"] = np.mean(sub_r1_AIM_ROI[:, 0:2], axis=1)
            ROI_raw_timecourses["Left_NAcc_r1"] = sub_r1_AIM_ROI[:, 1]
            ROI_raw_timecourses["Right_NAcc_r1"] = sub_r1_AIM_ROI[:, 0]
            ROI_raw_timecourses["Bilateral_AIns_r1"] = np.mean(sub_r1_AIM_ROI[:, 2:4], axis=1)
            ROI_raw_timecourses["Left_AIns_r1"] = sub_r1_AIM_ROI[:, 3]
            ROI_raw_timecourses["Right_AIns_r1"] = sub_r1_AIM_ROI[:, 2]
            ROI_raw_timecourses["Bilateral_MPFC_r1"] = np.mean(sub_r1_AIM_ROI[:, 4:6], axis=1)
            ROI_raw_timecourses["Left_MPFC_r1"] = sub_r1_AIM_ROI[:, 5]
            ROI_raw_timecourses["Right_MPFC_r1"] = sub_r1_AIM_ROI[:, 4]
            ROI_raw_timecourses["Bilateral_V1_r1"] = np.mean(sub_r1_AIM_ROI[:, 6:8], axis=1)
            ROI_raw_timecourses["Left_V1_r1"] = sub_r1_AIM_ROI[:, 6]
            ROI_raw_timecourses["Right_V1_r1"] = sub_r1_AIM_ROI[:, 7]
        else:
            ROI_raw_timecourses["Bilateral_NAcc_r1"] = np.mean(sub_r1_AIM_ROI[:, 0:2], axis=1)
            ROI_raw_timecourses["Left_NAcc_r1"] = sub_r1_AIM_ROI[:, 1]
            ROI_raw_timecourses["Right_NAcc_r1"] = sub_r1_AIM_ROI[:, 0]
            ROI_raw_timecourses["Bilateral_AIns_r1"] = np.mean(sub_r1_AIM_ROI[:, 2:4], axis=1)
            ROI_raw_timecourses["Left_AIns_r1"] = sub_r1_AIM_ROI[:, 3]
            ROI_raw_timecourses["Right_AIns_r1"] = sub_r1_AIM_ROI[:, 2]
            ROI_raw_timecourses["Bilateral_MPFC_r1"] = np.mean(sub_r1_AIM_ROI[:, 4:6], axis=1)
            ROI_raw_timecourses["Left_MPFC_r1"] = sub_r1_AIM_ROI[:, 5]
            ROI_raw_timecourses["Right_MPFC_r1"] = sub_r1_AIM_ROI[:, 4]



    # Apply for second run.
    if((sub_motion[2] < 1) and (sub_motion[3] < 1)):

        # Apply function to get the percent signal change from each ROI timecourse.
        sub_r2_AIM_ROI = masker_AIM_ROI_r2.fit_transform(sub_run2_func_path, confounds=confounds2)

        # Create dictionary variable to store arrays with time series arrays for each trailer.
        run2_timeseries = {}

        r2_keys = list(run2_onsets.keys())

        # Loop through each traile and get its corresponding ROI timecourse
        for id in range(len(r2_keys)):
            run2_timeseries[r2_keys[id]] = {
                "Bilateral_NAcc": np.mean(sub_r2_AIM_ROI[run2_onsets[r2_keys[id]].astype(bool)][:, 0:2], axis=1),
                "Left_NAcc": sub_r2_AIM_ROI[run2_onsets[r2_keys[id]].astype(bool)][:, 1],
                "Right_NAcc": sub_r2_AIM_ROI[run2_onsets[r2_keys[id]].astype(bool)][:, 0],
                "Bilateral_AIns": np.mean(sub_r2_AIM_ROI[run2_onsets[r2_keys[id]].astype(bool)][:, 2:4], axis=1),
                "Left_AIns": sub_r2_AIM_ROI[run2_onsets[r2_keys[id]].astype(bool)][:, 3],
                "Right_AIns": sub_r2_AIM_ROI[run2_onsets[r2_keys[id]].astype(bool)][:, 2],
                "Bilateral_MPFC": np.mean(sub_r2_AIM_ROI[run2_onsets[r2_keys[id]].astype(bool)][:, 4:6], axis=1),
                "Left_MPFC": sub_r2_AIM_ROI[run2_onsets[r2_keys[id]].astype(bool)][:, 5],
                "Right_MPFC": sub_r2_AIM_ROI[run2_onsets[r2_keys[id]].astype(bool)][:, 4]}
        
        # This code gives me the un-trimmed timecourses for each ROI.
        if(type=="psc"):
            ROI_raw_timecourses["Bilateral_NAcc_r2"] = np.mean(sub_r2_AIM_ROI[:, 0:2], axis=1)
            ROI_raw_timecourses["Left_NAcc_r2"] = sub_r2_AIM_ROI[:, 1]
            ROI_raw_timecourses["Right_NAcc_r2"] = sub_r2_AIM_ROI[:, 0]
            ROI_raw_timecourses["Bilateral_AIns_r2"] = np.mean(sub_r2_AIM_ROI[:, 2:4], axis=1)
            ROI_raw_timecourses["Left_AIns_r2"] = sub_r2_AIM_ROI[:, 3]
            ROI_raw_timecourses["Right_AIns_r2"] = sub_r2_AIM_ROI[:, 2]
            ROI_raw_timecourses["Bilateral_MPFC_r2"] = np.mean(sub_r2_AIM_ROI[:, 4:6], axis=1)
            ROI_raw_timecourses["Left_MPFC_r2"] = sub_r2_AIM_ROI[:, 5]
            ROI_raw_timecourses["Right_MPFC_r2"] = sub_r2_AIM_ROI[:, 4]
            ROI_raw_timecourses["Bilateral_V1_r2"] = np.mean(sub_r2_AIM_ROI[:, 6:8], axis=1)
            ROI_raw_timecourses["Left_V1_r2"] = sub_r2_AIM_ROI[:, 6]
            ROI_raw_timecourses["Right_V1_r2"] = sub_r2_AIM_ROI[:, 7]
        else:
            ROI_raw_timecourses["Bilateral_NAcc_r2"] = np.mean(sub_r2_AIM_ROI[:, 0:2], axis=1)
            ROI_raw_timecourses["Left_NAcc_r2"] = sub_r2_AIM_ROI[:, 1]
            ROI_raw_timecourses["Right_NAcc_r2"] = sub_r2_AIM_ROI[:, 0]
            ROI_raw_timecourses["Bilateral_AIns_r2"] = np.mean(sub_r2_AIM_ROI[:, 2:4], axis=1)
            ROI_raw_timecourses["Left_AIns_r2"] = sub_r2_AIM_ROI[:, 3]
            ROI_raw_timecourses["Right_AIns_r2"] = sub_r2_AIM_ROI[:, 2]
            ROI_raw_timecourses["Bilateral_MPFC_r2"] = np.mean(sub_r2_AIM_ROI[:, 4:6], axis=1)
            ROI_raw_timecourses["Left_MPFC_r2"] = sub_r2_AIM_ROI[:, 5]
            ROI_raw_timecourses["Right_MPFC_r2"] = sub_r2_AIM_ROI[:, 4]

    ## 6) Merge timecourses and events data. 
    if((sub_motion[0] < 1) and (sub_motion[2] < 1)): 
        all_timeseries = merge_dictionaries(run1_timeseries, run2_timeseries)   
        sub_all_p_events = pd.concat([sub_run1_p_events, sub_run2_p_events], ignore_index=True)
        print("Participant " + participant_num + " has acceptable data for both runs.")
    elif((sub_motion[0] < 1) and (sub_motion[2] > 1)): 
        all_timeseries = run1_timeseries
        sub_all_p_events = sub_run1_p_events
        print("Participant " + participant_num + " has acceptable data for run 1 only.")
    elif((sub_motion[0] > 1) and (sub_motion[2] < 1)):
        all_timeseries = run2_timeseries
        sub_all_p_events = sub_run2_p_events
        print("Participant " + participant_num + " has acceptable data for run 2 only.")
    else:
        print("Participant " + participant_num + " has unacceptable data for both runs.")
        all_timeseries = {}
        sub_all_p_events = []
        
    return all_timeseries, sub_all_p_events, ROI_raw_timecourses



# Get neural activation time courses. 

In [78]:
# Define each subject motion parameters from fMRIprep.
# The first value is the max framewise displacement, the second is the mean framewise displacement.
# Two values per run. 
sub01_motion = [0.911, 0.095, 0.571, 0.077]
sub02_motion = [0.663, 0.073, 0.734, 0.108]
sub03_motion = [0.456, 0.048, 0.300, 0.049]
sub04_motion = [0.607, 0.080, 0.519, 0.076]


In [79]:
"""
Default set-up: 
    Confounds: "white_matter", "csf", "csf_wm", "framewise_displacement", "dvars", "rmsd", "tcompcor"
    "rotation_x", "rotation_y", "rotation_z", "translation_x", "translation_y", "translation_z",  "non-steady-stateX".
    Detrend: False
    Hemodynamic response function lag: 4 seconds
""" 

sub01_ROIs_timecourse, sub01_events_df, sub01_ROIs_timecourse_raw = getROIs_timecourse("01", "psc", sub01_motion) # Max-Mean FD -> Run 1: (0.911 , 0.095) Run 2: (0.571, 0.077)
sub02_ROIs_timecourse, sub02_events_df, sub02_ROIs_timecourse_raw = getROIs_timecourse("02", "psc", sub02_motion) # Max-Mean FD -> Run 1: (0.663 , 0.073) Run 2: (0.734, 0.108) 
sub03_ROIs_timecourse, sub03_events_df, sub03_ROIs_timecourse_raw = getROIs_timecourse("03", "psc", sub03_motion) # Max-Mean FD -> Run 1: (0.456 , 0.048) Run 2: (0.300, 0.049) 
sub04_ROIs_timecourse, sub04_events_df, sub04_ROIs_timecourse_raw = getROIs_timecourse("04", "psc", sub04_motion) # Max-Mean FD -> Run 1: (0.607 , 0.080) Run 2: (0.519, 0.076)
sub08_ROIs_timecourse, sub08_events_df, sub08_ROIs_timecourse_raw = getROIs_timecourse("08", "psc", sub04_motion) # Max-Mean FD -> Run 1: (0.607 , 0.080) Run 2: (0.519, 0.076) 
 


Getting ROI timecourse for participant: 01 and type: psc
Participant 01 has acceptable data for both runs.
Getting ROI timecourse for participant: 02 and type: psc
Participant 02 has acceptable data for both runs.
Getting ROI timecourse for participant: 03 and type: psc
Participant 03 has acceptable data for both runs.
Getting ROI timecourse for participant: 04 and type: psc
Participant 04 has acceptable data for both runs.
Getting ROI timecourse for participant: 08 and type: psc
Participant 08 has acceptable data for both runs.


In [80]:
# Get average timecourse for each ROI.
all_bNAcc_r1_array = np.vstack((sub01_ROIs_timecourse_raw["Bilateral_NAcc_r1"], sub02_ROIs_timecourse_raw["Bilateral_NAcc_r1"], sub03_ROIs_timecourse_raw["Bilateral_NAcc_r1"], sub04_ROIs_timecourse_raw["Bilateral_NAcc_r1"],
                                sub08_ROIs_timecourse_raw["Bilateral_NAcc_r1"]))
all_bNAcc_r1_avg = np.mean(all_bNAcc_r1_array, axis=0)

all_bAIns_r1_array = np.vstack((sub01_ROIs_timecourse_raw["Bilateral_AIns_r1"], sub02_ROIs_timecourse_raw["Bilateral_AIns_r1"], sub03_ROIs_timecourse_raw["Bilateral_AIns_r1"], sub04_ROIs_timecourse_raw["Bilateral_AIns_r1"],
                                sub08_ROIs_timecourse_raw["Bilateral_AIns_r1"]))
all_bAIns_r1_avg = np.mean(all_bAIns_r1_array, axis=0)

all_bMPFC_r1_array = np.vstack((sub01_ROIs_timecourse_raw["Bilateral_MPFC_r1"], sub02_ROIs_timecourse_raw["Bilateral_MPFC_r1"], sub03_ROIs_timecourse_raw["Bilateral_MPFC_r1"], sub04_ROIs_timecourse_raw["Bilateral_MPFC_r1"],
                                sub08_ROIs_timecourse_raw["Bilateral_MPFC_r1"]))
all_bMPFC_r1_avg = np.mean(all_bMPFC_r1_array, axis=0)

all_bNAcc_r2_array = np.vstack((sub01_ROIs_timecourse_raw["Bilateral_NAcc_r2"], sub02_ROIs_timecourse_raw["Bilateral_NAcc_r2"], sub03_ROIs_timecourse_raw["Bilateral_NAcc_r2"], sub04_ROIs_timecourse_raw["Bilateral_NAcc_r2"],
                                sub08_ROIs_timecourse_raw["Bilateral_NAcc_r2"]))
all_bNAcc_r2_avg = np.mean(all_bNAcc_r2_array, axis=0)

all_bAIns_r2_array = np.vstack((sub01_ROIs_timecourse_raw["Bilateral_AIns_r2"], sub02_ROIs_timecourse_raw["Bilateral_AIns_r2"], sub03_ROIs_timecourse_raw["Bilateral_AIns_r2"], sub04_ROIs_timecourse_raw["Bilateral_AIns_r2"],
                                sub08_ROIs_timecourse_raw["Bilateral_AIns_r2"]))
all_bAIns_r2_avg = np.mean(all_bAIns_r2_array, axis=0)

all_bMPFC_r2_array = np.vstack((sub01_ROIs_timecourse_raw["Bilateral_MPFC_r2"], sub02_ROIs_timecourse_raw["Bilateral_MPFC_r2"], sub03_ROIs_timecourse_raw["Bilateral_MPFC_r2"], sub04_ROIs_timecourse_raw["Bilateral_MPFC_r2"],
                                sub08_ROIs_timecourse_raw["Bilateral_MPFC_r2"]))
all_bMPFC_r2_avg = np.mean(all_bMPFC_r2_array, axis=0)



In [81]:
# Compute correlations between each roi.

# NAcc & MPFC 
bNAcc_bMPFC_r1 = pearsonr(all_bNAcc_r1_avg, all_bMPFC_r1_avg)
bNAcc_bMPFC_r2 = pearsonr(all_bNAcc_r2_avg, all_bMPFC_r2_avg)
print("Run 1 NAcc and MPFC have a correlation " + str(bNAcc_bMPFC_r1[0]) + " with a p-value of " + str(bNAcc_bMPFC_r1[1]))
#print("Run 2 NAcc and MPFC have a correlation " + str(bNAcc_bMPFC_r2[0]) + " with a p-value of " + str(bNAcc_bMPFC_r2[1]))

# NAcc & AIns
bNAcc_bAIns_r1 = pearsonr(all_bNAcc_r1_avg, all_bAIns_r1_avg)
bNAcc_bAIns_r2 = pearsonr(all_bNAcc_r2_avg, all_bAIns_r2_avg)
print("\nRun 1 NAcc and AIns have a correlation " + str(bNAcc_bAIns_r1[0]) + " with a p-value of " + str(bNAcc_bAIns_r1[1]))
#print("Run 2 NAcc and AIns have a correlation " + str(bNAcc_bAIns_r2[0]) + " with a p-value of " + str(bNAcc_bAIns_r2[1]))

# MPFC & AIns
bMPFC_bAIns_r1 = pearsonr(all_bMPFC_r1_avg, all_bAIns_r1_avg)
bMPFC_bAIns_r2 = pearsonr(all_bMPFC_r2_avg, all_bAIns_r2_avg)
print("\nRun 1 MPFC and AIns have a correlation " + str(bMPFC_bAIns_r1[0]) + " with a p-value of " + str(bMPFC_bAIns_r1[1]))
#print("Run 2 MPFC and AIns have a correlation " + str(bMPFC_bAIns_r2[0]) + " with a p-value of " + str(bMPFC_bAIns_r2[1]))




Run 1 NAcc and MPFC have a correlation 0.2772310943366475 with a p-value of 4.255268758197753e-08

Run 1 NAcc and AIns have a correlation 0.39414353026779597 with a p-value of 1.6892182517886263e-15

Run 1 MPFC and AIns have a correlation 0.357633033760682 with a p-value of 7.576024020636746e-13


In [91]:
"""
Default set-up: 
    Confounds: "white_matter", "csf", "csf_wm", "framewise_displacement", "dvars", "rmsd", "tcompcor"
    "rotation_x", "rotation_y", "rotation_z", "translation_x", "translation_y", "translation_z",  "non-steady-stateX".
    Detrend: True
    Hemodynamic response function lag: 4 seconds
""" 

sub01_ROIs_timecourse, sub01_events_df, sub01_ROIs_timecourse_raw_detrended = getROIs_timecourse_detrend("01", "psc", sub01_motion) # Max-Mean FD -> Run 1: (0.911 , 0.095) Run 2: (0.571, 0.077)
sub02_ROIs_timecourse, sub02_events_df, sub02_ROIs_timecourse_raw_detrended = getROIs_timecourse_detrend("02", "psc", sub02_motion) # Max-Mean FD -> Run 1: (0.663 , 0.073) Run 2: (0.734, 0.108) 
sub03_ROIs_timecourse, sub03_events_df, sub03_ROIs_timecourse_raw_detrended = getROIs_timecourse_detrend("03", "psc", sub03_motion) # Max-Mean FD -> Run 1: (0.456 , 0.048) Run 2: (0.300, 0.049) 
sub04_ROIs_timecourse, sub04_events_df, sub04_ROIs_timecourse_raw_detrended = getROIs_timecourse_detrend("04", "psc", sub04_motion) # Max-Mean FD -> Run 1: (0.607 , 0.080) Run 2: (0.519, 0.076) 
sub08_ROIs_timecourse, sub08_events_df, sub08_ROIs_timecourse_raw_detrended = getROIs_timecourse_detrend("08", "psc", sub04_motion) # Max-Mean FD -> Run 1: (0.607 , 0.080) Run 2: (0.519, 0.076)

Getting ROI timecourse for participant: 01 and type: psc
Participant 01 has acceptable data for both runs.
Getting ROI timecourse for participant: 02 and type: psc
Participant 02 has acceptable data for both runs.
Getting ROI timecourse for participant: 03 and type: psc
Participant 03 has acceptable data for both runs.
Getting ROI timecourse for participant: 04 and type: psc
Participant 04 has acceptable data for both runs.
Getting ROI timecourse for participant: 08 and type: psc
Participant 08 has acceptable data for both runs.


In [92]:
# Get average timecourse for each ROI.
all_bNAcc_r1_detrended_array = np.vstack((sub01_ROIs_timecourse_raw_detrended["Bilateral_NAcc_r1"], sub02_ROIs_timecourse_raw_detrended["Bilateral_NAcc_r1"], sub03_ROIs_timecourse_raw_detrended["Bilateral_NAcc_r1"], sub04_ROIs_timecourse_raw_detrended["Bilateral_NAcc_r1"],
                                          sub08_ROIs_timecourse_raw_detrended["Bilateral_NAcc_r1"]))
all_bNAcc_r1_detrended_avg = np.mean(all_bNAcc_r1_detrended_array, axis=0)

all_bAIns_r1_detrended_array = np.vstack((sub01_ROIs_timecourse_raw_detrended["Bilateral_AIns_r1"], sub02_ROIs_timecourse_raw_detrended["Bilateral_AIns_r1"], sub03_ROIs_timecourse_raw_detrended["Bilateral_AIns_r1"], sub04_ROIs_timecourse_raw_detrended["Bilateral_AIns_r1"],
                                          sub08_ROIs_timecourse_raw_detrended["Bilateral_AIns_r1"]))
all_bAIns_r1_detrended_avg = np.mean(all_bAIns_r1_detrended_array, axis=0)

all_bMPFC_r1_detrended_array = np.vstack((sub01_ROIs_timecourse_raw_detrended["Bilateral_MPFC_r1"], sub02_ROIs_timecourse_raw_detrended["Bilateral_MPFC_r1"], sub03_ROIs_timecourse_raw_detrended["Bilateral_MPFC_r1"], sub04_ROIs_timecourse_raw_detrended["Bilateral_MPFC_r1"],
                                          sub08_ROIs_timecourse_raw_detrended["Bilateral_MPFC_r1"]))
all_bMPFC_r1_detrended_avg = np.mean(all_bMPFC_r1_detrended_array, axis=0)



In [93]:
# Compute correlations between each roi.

# NAcc & MPFC 
bNAcc_bMPFC_r1_detrended = pearsonr(all_bNAcc_r1_detrended_avg, all_bMPFC_r1_detrended_avg)
print("Run 1 NAcc and MPFC have a correlation " + str(bNAcc_bMPFC_r1_detrended[0]) + " with a p-value of " + str(bNAcc_bMPFC_r1_detrended[1]))

# NAcc & AIns
bNAcc_bAIns_r1_detrended = pearsonr(all_bNAcc_r1_detrended_avg, all_bAIns_r1_detrended_avg)
print("Run 1 NAcc and AIns have a correlation " + str(bNAcc_bAIns_r1_detrended[0]) + " with a p-value of " + str(bNAcc_bAIns_r1_detrended[1]))

# MPFC & AIns
bMPFC_bAIns_r1_detrended = pearsonr(all_bMPFC_r1_detrended_avg, all_bAIns_r1_detrended_avg)
print("Run 1 MPFC and AIns have a correlation " + str(bMPFC_bAIns_r1_detrended[0]) + " with a p-value of " + str(bMPFC_bAIns_r1_detrended[1]))




Run 1 NAcc and MPFC have a correlation 0.3516110815258614 with a p-value of 1.9293754094885433e-12
Run 1 NAcc and AIns have a correlation 0.4075357489024547 with a p-value of 1.476249732498815e-16
Run 1 MPFC and AIns have a correlation 0.32051828404854243 with a p-value of 1.7703619570055808e-10
